# Kaggle - [Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/碩一下/Machine Learning/Homework_5') 
os.listdir()

In [6]:
!git clone https://github.com/sueshow/Kaggle.git

Cloning into 'Kaggle'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
#若需要重新下載資料，先執行下面的語法：
#!rm -r /content/Kaggle

## 使用模型：Support Vector Machine (SVM) Algorithm

### Exploratory Analysis

* There are thirteen features and one target as below:
  * age: The person's age in years
  * sex: The person's sex (1 = male, 0 = female)
  * cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
  * trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)
  * chol: The person's cholesterol measurement in mg/dl
  * fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
  * restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
  * thalach: The person's maximum heart rate achieved
  * exang: Exercise induced angina (1 = yes; 0 = no)
  * oldpeak: ST depression induced by exercise relative to rest
  * slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
  * ca: The number of major vessels (0-3)
  * thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
  * target: Heart disease (0 = no, 1 = yes)

## Imporrt Package

In [7]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# GridSearchCV
#from sklearn.metrics import classification_report
#from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.naive_bayes import GaussianNB, BernoulliNB

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from xgboost import XGBClassifier

from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

### Read the data and show the first 5 rows

In [3]:
df = pd.read_csv('https://drive.google.com/drive/folders/14pUuY7P4J3vvAyNF-I2ZwVGv5wk4TZep/heart.csv')
df.head()

HTTPError: ignored

### Check for missing values and basic informations

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().any()

In [ ]:
df.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                           .background_gradient(subset=['std'], cmap='Reds')\
                           .background_gradient(subset=['50%'], cmap='coolwarm')

* Cholesterol has the highest mean value of 246.26 & also he highest standard deviation of 51.83

### Basic Plots

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), annot=True)

* slope and old peak has high negative correlation of -0.58. This means if slope value increases then old peak will decrease and vice versa.
* Target and cp(chest pain) has the highest positive correlation of 0.43.

In [ ]:
plt.figure(figsize=(20,15))
sns.set_theme(style='dark')
plt.subplot(2,3,1)
sns.countplot(data=df, x='fbs', hue='target')
plt.subplot(2,3,2)
sns.countplot(data=df, x='restecg', hue='target')
plt.subplot(2,3,3)
sns.countplot(data=df, x='slope', hue='target')
plt.subplot(2,3,4)
sns.countplot(data=df, x='ca', hue='target')
plt.subplot(2,3,5)
sns.countplot(data=df, x='exang', hue='target')
plt.subplot(2,3,6)
sns.countplot(data=df, x='thal', hue='target')
plt.show()

* This plot depicts an important relationship.
  * For example, in the 1st subplot of "fbs", it says that when fbs is zero, the target value zero is about 120 and target value 1 is 140, again when fbs is one, the target value for zero is about 20 and target value for one is just above 20.

In [ ]:
df.hist(figsize=(20,16))
plt.show()

* This also depicts the counts of each value of each column. Here we can see the imblances in data too.
  * For example the fbs column has around 250 zero values, and about 20 one values.

In [ ]:
plt.figure(figsize=(13,13))

sns.set_theme(style='darkgrid')
plt.subplot(2,3,1)
sns.boxplot(x='thal', data=df)
plt.subplot(2,3,2)
sns.boxplot(x='oldpeak', data=df)
plt.subplot(2,3,3)
sns.boxplot(x='thalach', data=df)
plt.subplot(2,3,4)
sns.boxplot(x='chol', data=df)
plt.subplot(2,3,5)
sns.boxplot(x='trestbps', data=df)
plt.subplot(2,3,6)
sns.boxplot(x='age', data=df)
plt.show()

In [ ]:
import plotly.express as px

fig = px.box(df, x='trestbps')
fig.show()

### Scaling after train_test_split

In [ ]:
X = df.drop(['target'], axis = 1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

#transpose matrices
x_train.T = x_train 
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

###  Model

In [ ]:
svm = SVC(random_state = 1)
svm.fit(x_train.T, y_train.T)

acc = svm.score(x_test.T,y_test.T)*100
accuracies['SVM'] = acc
print("Test Accuracy of SVM Algorithm: {:.2f}%".format(acc))

In [ ]:
# Predicted values
y_head_svm = svm.predict(x_test.T)

In [ ]:
from sklearn.metrics import confusion_matrix
cm_svm = confusion_matrix(y_test,y_head_svm)

In [ ]:
plt.figure()
plt.title("Support Vector Machine Confusion Matrix")
sns.heatmap(cm_svm,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train.T, y_train.T)

acc = dtc.score(x_test.T, y_test.T)*100
accuracies['Decision Tree'] = acc
print("Decision Tree Test Accuracy {:.2f}%".format(acc))

In [ ]:
svm_model = SVC(kernel='rbf', class_weight='balanced',probability=True)

param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

grid_search = GridSearchCV(estimator=svm_model,param_grid=param_grid, verbose=10,cv=10)
grid_search.fit(X_train,Y_train)

print("best params:{}".format(grid_search.best_params_, grid_search.best_score_))
final_model = grid_search.best_estimator_

In [ ]:
# Y_pred = final_model.predict(X_test)
# print(classification_report(Y_test, Y_pred))
final_model.score(X_test,Y_test)

In [ ]:
models = []
models.append(['RidgeClassifier',RidgeClassifier()])
models.append(['XGBClassifier',XGBClassifier(use_label_encoder=False,objective='binary:logistic',random_state=0,eval_metric='logloss')])
models.append(['Logistic Regression',LogisticRegression(random_state=0)])
models.append(['SVM',SVC(random_state=0)])
models.append(['KNeigbors',KNeighborsClassifier()])
models.append(['GaussianNB',GaussianNB()])
models.append(['BernoulliNB',BernoulliNB()])
models.append(['DecisionTree',DecisionTreeClassifier(random_state=0)])
models.append(['RandomForest',RandomForestClassifier(random_state=0)])
models.append(['AdaBoostClassifier',AdaBoostClassifier()])
models.append(['MLPClassifier',MLPClassifier(random_state = 42, max_iter=1000)])
models.append(['ExtraTreesClassifier',ExtraTreesClassifier()])
models.append(['CatBoostClassifier', CatBoostClassifier(eval_metric = 'AUC', verbose = 0)])
models.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
models.append(['SGDClassifier',SGDClassifier()])

In [ ]:
lst_1 = []
for m in range(len(models)):
    lst_2 = []
    model = models[m][1]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracies = cross_val_score(estimator= model, X = X_train,y = y_train, cv=10)

# k-fOLD Validation
    roc = roc_auc_score(y_test,y_pred)
    
    print(models[m][0],':')
    print(cm)
    print('Accuracy Score: ',accuracy_score(y_test,y_pred))
    print('')
    print('K-Fold Validation Mean Accuracy: {:.2f} %'.format(accuracies.mean()*100))
    print('')
    print('ROC AUC Score: {:.2f}'.format(roc))
    print('-'*40)
    print('')
    lst_2.append(models[m][0])
    lst_2.append(accuracy_score(y_test,y_pred)*100)
    lst_2.append(accuracies.mean()*100)
    lst_2.append(roc)
    lst_1.append(lst_2)

In [ ]:
df2 = pd.DataFrame(lst_1,columns=['Model','Accuracy','K-Fold Mean Accuracy','ROC_AUC'])

df2.sort_values(by=['ROC_AUC'],inplace=True,ascending=False)
df2

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.barplot(x='ROC_AUC',y='Model',data=df2,color='r')
plt.title('Model Comparison');

In [ ]:
grid_models = [
               (KNeighborsClassifier(),[{'n_neighbors':np.arange(1, 100), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}]), 
               (DecisionTreeClassifier(),[{'criterion':['gini','entropy'],'max_depth':np.arange(1, 50), 'min_samples_leaf':[1,2,4]}]), 
               (RandomForestClassifier(),[{'n_estimators':[100,150,200],'criterion':['gini','entropy'], 'min_samples_leaf':[2, 10, 30]}]),
               (MLPClassifier(max_iter = 1000),[{'solver':['lbfgs', 'sgd', 'adam'], 'learning_rate' :['constant', 'invscaling', 'adaptive']}]), 
               (RidgeClassifier(),[{'alpha':[0.1,0.5,1], 'solver':['auto', 'svd', 'cholesky']}]),
               (GaussianNB(),[{'var_smoothing': np.logspace(0,-9, num=100)}]),
               (XGBClassifier(use_label_encoder = False), [{'learning_rate': [0.01, 0.05, 0.1], 'eval_metric': ['error', 'logloss']}])
               ]

In [ ]:
for i,j in grid_models:
    grid = GridSearchCV(estimator=i,param_grid = j, scoring = 'roc_auc',cv = 5)
    grid.fit(X_train,y_train)
    best_score = grid.best_score_
    best_param = grid.best_params_
    print(' {}: \n Best score: {:.1f} %'.format(i,best_score*100))
    print('')
    print('-'*25)
    print('')

* Top performing model is KNeighborsClassifier and then MLPClassifier.

## 參考資料

* [Kaggle](https://www.kaggle.com/cdabakoglu/heart-disease-classifications-machine-learning)